# 3º Hackdays CDS

# 0.0 Import libs

In [1]:
import pandas                as pd
from sklearn.preprocessing   import LabelEncoder
from sklearn                 import model_selection as ms
from xgboost                 import XGBClassifier
from sklearn.metrics         import f1_score
from sklearn.model_selection import cross_val_score

# 0.1 Load Data

In [2]:
# Carregar dataset treino kaggle
df = pd.read_csv('../data/train.csv')

In [3]:
# Listar dataset
df

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,5155,29,157.500279,157.98,32624.67754,237.739275,32.407322,100,4,4,24,15,1,14,Sim,Não,Negar
9496,11977,1237,497.714090,805.43,49024.15700,118.883577,23.632195,0,4,7,10,11,2,3,Não,Não,Negar
9497,9278,47,306.557684,2250.42,42200.88978,134.201478,34.623576,4,4,4,13,8,4,24,Não,Não,Negar
9498,2525,42,209.870718,505.37,35391.32289,245.498881,28.758818,3,7,6,24,13,3,11,Sim,Não,Negar


# Steps Jumped

In [4]:
# Fillout NA
# Change Types
# Estatística descritiva básica 
# Feature engeneering
# Filtragem de variáveis inúteis
# Análise Exploratória dos Dados

# Data Preparation

In [5]:
# Dividir dataset entre Treino (80%) e Teste (20%) 
X = df.drop('limite_adicional', axis = 1).copy()
y = df['limite_adicional'].copy()

x_train, x_test, y_train, y_test = ms.train_test_split(X, y, stratify = y, test_size = 0.20, random_state = 42)

In [6]:
# df2 será usado para splitar Treino e Validação
df2 = pd.concat([x_train, y_train], axis = 1)

# Dividir dataset entre Treino (90%) e Validação (10%)
X = df2.drop(['limite_adicional'], axis=1).copy()
y = df2['limite_adicional'].copy()

x_train, x_val, y_train, y_val = ms.train_test_split(X, y, stratify = y, test_size=0.10, random_state = 42)

In [7]:
# Dados de treino
print('x_train -> ',x_train.shape)
print('y_train -> ',y_train.shape)
# Dados de validação
print('x_val -> ',x_val.shape)
print('y_val -> ',y_val.shape)
# Dados de teste
print('x_test -> ',x_test.shape)
print('y_test -> ',y_test.shape)

x_train ->  (6840, 16)
y_train ->  (6840,)
x_val ->  (760, 16)
y_val ->  (760,)
x_test ->  (1900, 16)
y_test ->  (1900,)


In [8]:
# Preparação dos dados
# Encodar features categóricas de treino
le_inv_ext = LabelEncoder()
le_pes_pol = LabelEncoder()
x_train['investe_exterior'] = le_inv_ext.fit_transform(x_train['investe_exterior'])
x_train['pessoa_polit_exp'] = le_pes_pol.fit_transform(x_train['pessoa_polit_exp'])

# Encodar var resposta de treino
lim_adi_dict = {'Negar':0, 'Conceder':1}
y_train = y_train.map(lim_adi_dict)

# Encodar features categóricas de validação
x_val['investe_exterior']   = le_inv_ext.transform(x_val['investe_exterior'])
x_val['pessoa_polit_exp']   = le_pes_pol.transform(x_val['pessoa_polit_exp'])

# Encodar var resposta de validação
y_val = y_val.map(lim_adi_dict)

# Machine Learning Models

In [9]:
# Aplicação de modelos 
# Treinar XGBoost Classifier
# Model definition
model_xgboost = XGBClassifier(random_state=42)
# Model training
model_xgboost.fit(x_train, y_train)
# Model prediction
yhat_xgboost = model_xgboost.predict(x_val)

In [10]:
# Avaliação de modelos 
# Validar modelo com F1 Score (dados validação)
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(f1_score(y_val, yhat_xgboost, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados validação): 0.869


In [11]:
# Validar modelo com F1 Score, no método cross validation (dados validação) 
print(f"K-folds: {cross_val_score(model_xgboost, x_val, y_val, scoring='f1', cv = 10)}") #'F1' métrica F1 Score p/ classificação binary
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(cross_val_score(model_xgboost, x_val, y_val, scoring='f1_weighted', cv = 10).mean() ,3)}")
# Performance manteve-se próxima com cross validation

K-folds: [0.63636364 0.44444444 0.33333333 0.54545455 0.6        0.33333333
 0.63636364 0.76190476 0.54545455 0.42105263]
XGBoost - Média ponderada com F1 Score (dados validação): 0.861


In [12]:
# Testar modelos
# Antes, juntar dados de treino + validação, formando um novo dataset de treino.
x_train_full = pd.concat([x_train, x_val])
y_train_full = pd.concat([y_train, y_val])

In [13]:
# Encodar features categóricas de teste
x_test['investe_exterior']   = le_inv_ext.transform(x_test['investe_exterior'])
x_test['pessoa_polit_exp']   = le_pes_pol.transform(x_test['pessoa_polit_exp'])

# Encodar var resposta de teste
y_test = y_test.map(lim_adi_dict)

In [14]:
# Treinar nova XGBoost, com o novo dataset treino
# Model definition
model_xgboost_full = XGBClassifier(random_state=42)
# Model training
model_xgboost_full.fit(x_train_full, y_train_full)
# Model prediction
# Passar os dados de teste separados anteriormente
yhat_xgboost_full = model_xgboost_full.predict(x_test)

In [15]:
# Validar modelo com F1 Score (dados teste)
print(f"XGBoost - Média ponderada com F1 Score (dados teste): {round(f1_score(y_test, yhat_xgboost_full, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados teste): 0.865


In [16]:
# Resultados:
 # XGBoost - Média ponderada com F1 Score (dados validação): 0.86
 # XGBoost - Média ponderada com F1 Score (dados teste): 0.86
# O modelo apresenta boa capacidade de generalização (classificar dados inéditos). 

In [17]:
# Carregar dataset teste
df_ktest_raw = pd.read_csv('../data/test.csv')
x_ktest = df_ktest_raw.copy()
# Conferir dataset teste kaggle
x_ktest

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp
0,2,32,470.690627,605.03,42688.60400,104.291825,24.464031,1,2,4,3,4,2,6,Não,Não
1,5,25,288.605522,943.86,43545.94475,276.725394,39.797764,0,7,5,5,14,4,5,Não,Não
2,6,35,740.196090,548.20,82208.44752,98.674410,39.962685,0,4,1288,12,10,2,8,Não,Não
3,8,7586,298.192158,1704.18,41749.69050,162.441009,24.448063,3,8,7,30,11,5,15,Não,Não
4,10,49,341.554423,421.43,42381.95080,154.352781,29.519353,2,0,5,4,0,2,12,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,12484,48,417.139276,636.96,45532.40174,94.723661,25.382445,2,7,3,6,8,4,19,Sim,Não
2996,12487,43,720.344280,717.79,95347.46144,160.090877,27.777284,2,6,7,30,20,5,11,Sim,Não
2997,12489,44,874.910490,361.68,97517.21808,98.884010,43.307916,0,2,7,5,7,1,7,Não,Não
2998,12495,24,349.184777,678.57,46943.70275,131.988993,35.123923,4,4,4,11,4,3,3,Não,Não


In [18]:
# Encodar features categóricas de teste kaggle
x_ktest['investe_exterior'] = le_inv_ext.transform(x_ktest['investe_exterior'])
x_ktest['pessoa_polit_exp'] = le_pes_pol.transform(x_ktest['pessoa_polit_exp'])

In [19]:
# Fazer a previsão do score usando o modelo final treinado:
yhat_xgboost_ktest = model_xgboost_full.predict(x_ktest)

# Submission

In [20]:
# Preparação para submissão
# Criar dataset final
df_sub = pd.concat([df_ktest_raw['id_cliente'], pd.Series(yhat_xgboost_ktest, name='limite_adicional')], axis=1)

In [21]:
# Traduzir var resposta
lim_adi_dic_final = {0 : 'Negar', 1: 'Conceder'}
df_sub['limite_adicional'] = df_sub['limite_adicional'].map( lim_adi_dic_final )

In [22]:
# Visualizar arquivo final para submissão ao Kaggle
df_sub[:5]

,id_cliente,limite_adicional
0,2,Conceder
1,5,Negar
2,6,Conceder
3,8,Negar
4,10,Conceder


In [23]:
# Gerar export para submission no Kaggle
df_sub.to_csv('../data/baseline_kaggle.csv', index=False) 